In [25]:
import logging
import csv

from rest_api_client import RestApiClient
from rest_api_client import SonarQubeClient

In [ ]:
SONAR_PROJECT_KEY = "test"
OUTPUT_FILE = "duplication_report.csv"
TOKEN = "squ_a2a88c9e7a963709d524f15d5a65578ccf339b56"

client = RestApiClient("http://172.23.79.169:9000", log_level=logging.DEBUG)
client.set_auth_basic(TOKEN, "")
projects = client.get("/api/projects/search")
client.get("/api/measures/component")

In [ ]:
client.get(
    f"api/measures/component?component={SONAR_PROJECT_KEY}&metricKeys=bugs,vulnerabilities,code_smells,duplicated_blocks,functions,duplicated_lines_density"
)

In [ ]:
# 対象メトリクス
metrics = "duplicated_lines,duplicated_blocks,duplicated_files,duplicated_lines_density"

rows: list[dict[str, Any]] = []

# 各プロジェクトのメトリクス取得
for proj in projects["components"]:
    key = proj["key"]
    name = proj["name"]

    result = client.get("/api/measures/component", params={"component": key, "metricKeys": metrics})

    if not result or "component" not in result:
        continue

    # メトリクス整形
    values = {m["metric"]: m["value"] for m in result["component"]["measures"]}
    row = {
        "key": key,
        "name": name,
        "duplicated_lines": values.get("duplicated_lines", ""),
        "duplicated_blocks": values.get("duplicated_blocks", ""),
        "duplicated_files": values.get("duplicated_files", ""),
        "duplicated_lines_density": values.get("duplicated_lines_density", ""),
    }
    rows.append(row)

# CSV 出力
with open("sonarqube_duplication_report.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=rows[0].keys())
    writer.writeheader()
    writer.writerows(rows)

In [ ]:
metrics = "duplicated_lines, duplicated_blocks, duplicated_files, duplicated_lines_density"
result = client.get("/api/measures/component", params={"component": SONAR_PROJECT_KEY, "metricKeys": metrics})
values = {m["metric"]: m["value"] for m in result["component"]["measures"]}
rows: list[dict[str, Any]] = []
row = {
    "key": key,
    "name": name,
    "duplicated_lines": values.get("duplicated_lines", ""),
    "duplicated_blocks": values.get("duplicated_blocks", ""),
    "duplicated_files": values.get("duplicated_files", ""),
    "duplicated_lines_density": values.get("duplicated_lines_density", ""),
}
rows.append(row)
with open("sonarqube_duplication_report.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=rows[0].keys())
    writer.writeheader()
    writer.writerows(rows)

result

In [ ]:
response = client.get(
    "/api/measures/component_tree",
    params={
        "component": SONAR_PROJECT_KEY,
        "metricKeys": "duplicated_lines_density",
        "qualifiers": "FIL",
        "ps": 500,
        "p": 1,
    },
)

if not response or "components" not in response:
    print("データ取得に失敗しました")
    exit(1)

# データ抽出・CSV出力
with open(OUTPUT_FILE, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["key", "duplicated_lines_density"])  # ヘッダー

    for comp in response["components"]:
        key = comp.get("key", "")
        value = ""
        for m in comp.get("measures", []):
            if m["metric"] == "duplicated_lines_density":
                value = m.get("value", "")
                break
        writer.writerow([key, value])

print(f"書き出し完了 → {OUTPUT_FILE}")

In [31]:
rows: list[dict[str, Any]] = []

# 各プロジェクトのメトリクス取得
for proj in projects["components"]:
    key = proj["key"]
    name = proj["name"]

    result = client.get("/api/measures/component", params={"component": key, "metricKeys": metrics})

    if not result or "component" not in result:
        continue

    # メトリクス整形
    values = {m["metric"]: m["value"] for m in result["component"]["measures"]}
    row = {
        "key": key,
        "name": name,
        "duplicated_lines": values.get("duplicated_lines", ""),
        "duplicated_blocks": values.get("duplicated_blocks", ""),
        "duplicated_files": values.get("duplicated_files", ""),
        "duplicated_lines_density": values.get("duplicated_lines_density", ""),
    }
    rows.append(row)

# CSV 出力
with open("sonarqube_duplication_report.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=rows[0].keys())
    writer.writeheader()
    writer.writerows(rows)

In [ ]:
logging.basicConfig(level=logging.DEBUG)
sonar = SonarQubeClient(base_url="http://localhost:9000", token="squ_a2a88c9e7a963709d524f15d5a65578ccf339b56")

metrics = ["ncloc", "duplicated_lines_density", "complexity"]
sonar.fetch_project_metrics_to_csv(project_key="test", metrics=metrics, output_file="metrics.csv")
sonar.fetch_duplication_list_to_csv(project_key="test", output_file="duplications.csv")
sonar.close()

In [ ]:
response

In [ ]:
# データ取得
results = []
page = 1
page_size = 500

while True:
    response = client.get(
        "/api/components/tree",
        params={
            "component": SONAR_PROJECT_KEY,
            "metricKeys": "duplicated_lines,duplicated_lines_density",
            "qualifiers": "FIL",
            "ps": page_size,
            "p": page,
        }
    )

    if not response or "components" not in response:
        break

    for component in response["components"]:
        path = component.get("path")
        measures = {m["metric"]: m["value"] for m in component.get("measures", [])}
        duplicated_lines = measures.get("duplicated_lines", "0")
        duplicated_percent = measures.get("duplicated_lines_density", "0")

        results.append({
            "file": path,
            "duplicated_lines": duplicated_lines,
            "duplicated_percent": duplicated_percent,
        })

    if len(response["components"]) < page_size:
        break

    page += 1

# CSV 出力
with open("sonarqube_duplicated_lines.csv", "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["file", "duplicated_lines", "duplicated_percent"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(results)

print("CSV 出力完了：sonarqube_duplicated_lines.csv")

In [ ]:
endpoint = "/api/issues/search"
page = 1
page_size = 500
all_issues = []

while True:
    params = {
        "componentKeys": SONAR_PROJECT_KEY,
        "types": "CODE_SMELL",
        # "rules": "cpp:S124",  # 重複コードのルール (例: C/C++の場合)
        "rules": "python:S1871",
        "ps": page_size,
        "p": page,
    }
    result = client.get(endpoint, params=params)
    if not result:
        break

    issues = result.get("issues", [])
    if not issues:
        break

    all_issues.extend(issues)
    if len(issues) < page_size:
        break
    page += 1
all_issues